<a href="https://colab.research.google.com/github/dyou-wbg/meta_processing/blob/main/bert_50k_sample__char_based_distance_knn_deduping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from google.colab import drive
import csv
import os
import re
import pandas as pd
import numpy as np

directory = '/content/drive/MyDrive/finalMetaFile'

Mounted at /content/drive


In [2]:

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)


<ipython-input-2-1390d13c0cdd>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [3]:
df_english_titles = pd.read_csv('/content/drive/MyDrive/finalMetaFile/50k_sample.csv')

In [6]:
from sklearn.neighbors import KDTree
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


In [8]:
from string import printable
from collections import Counter

printable_dict = {c:i for i,c in enumerate(printable)}


def vectorize(title):
    cnt = Counter(title)
    array = [0 for _ in printable]+[0] # Last position is for nonprintable characters
    for c,n in cnt.items():
        if c in printable_dict:
            array[printable_dict[c]]=n
        else:
            array[-1]+=n # Update last position if it's not printable
    return np.array(array)

vects = []
for title in df_english_titles.title:
    vects.append(vectorize(title))

# benchmark 1: simple tf-idf and consine similarity

In [9]:
# took around 1 min
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_english_titles['title'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
%%time
num_titles = len(df_english_titles)
similar_titles = []

for i in range(num_titles):
    current_title = df_english_titles.iloc[i]['title']
    similarities = cosine_sim[i]

    for j in range(num_titles):
        if i != j:
            other_title = df_english_titles.iloc[j]['title']
            similarity_score = similarities[j]


            if similarity_score > 0.85  :  # Example threshold for high similarity
                print(f"{current_title} & {other_title} (score: {similarity_score:.4f})")

            if similar_titles:  # If any similar titles found
              all_similar_titles.append({"title": current_title,"similar_titles": similar_titles})

CENTRAL EUROPEAN REVIEW OF ECONOMICS AND MANAGEMENT & CENTRAL EUROPEAN REVIEW OF ECONOMICS AND MANAGEMENT (score: 1.0000)
Defense Briefs & Defense Briefs (score: 1.0000)
Relative Wage Variability: Monetary Policy and the Labor Market & Relative Wage Variability: Monetary Policy and the Labor Market (score: 1.0000)
Editor's introduction & Editor's Introduction (score: 1.0000)
Editor's introduction & Editor's Introduction (score: 1.0000)
Calcutta Diary & Calcutta Diary (score: 1.0000)
Calcutta Diary & Calcutta Diary (score: 1.0000)
Calcutta Diary & Calcutta Diary (score: 1.0000)
JURISPRUDENCE FINANCIÈRE: RÉPERTOIRE ANALYTIQUE & JURISPRUDENCE FINANCIÈRE: Répertoire analytique (score: 1.0000)
Could the Corporations Become Engines for International Cooperation? & COULD THE CORPORATIONS BECOME ENGINES FOR INTERNATIONAL COOPERATION (score: 1.0000)
Identifying the Monetary Transmission Mechanism Using Structural Breaks & Identifying the monetary transmission mechanism using structural breaks (

In [ ]:
all_similar_titles_tfidf = []

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_english_titles['title'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

num_titles = len(df_english_titles)
for i in range(num_titles):
    current_title = df_english_titles.iloc[i]['title']
    similarities = cosine_sim[i]
    similar_titles = []

    for j in range(num_titles):
        if i != j:  # Skip comparison with itself
            other_title = df_english_titles.iloc[j]['title']
            similarity_score = similarities[j]

            # Actions based on similarity score:
            if similarity_score > 0.85:
                print(f"{current_title} is highly similar to {other_title} (score: {similarity_score:.4f})")
                similar_titles.append(other_title)  # Add to similar titles list

    if similar_titles:  # If any similar titles found, store in all_similar_titles
        all_similar_titles_tfidf.append({"title": current_title, "similar_titles": similar_titles})

all_similar_titles_tfidf=pd.DataFrame(all_similar_titles_tfidf)
all_similar_titles_tfidf.to_csv(os.path.join(directory, '50k_sample_similar_titles_bytfidf_consineSim.csv'), index=False)

# benchmark 2 : BERT

In [ ]:
pip install sentence-transformers

In [ ]:
from sklearn.neighbors import KDTree
from transformers import BertModel, BertTokenizer
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2')


def get_encoded_vectors(titles):
  encoded_inputs = model.encode(titles)
  return encoded_inputs

encoded_vects = get_encoded_vectors(df_english_titles['title'])

tree = KDTree(encoded_vects, leaf_size=10, metric='cosine')

In [ ]:



# Function to find and save similar titles using cosine similarity
def find_similar_titles(df_titles, threshold=0.85):
  similar_titles_dict = {}
  for i in range(len(df_titles)):
    current_title = df_titles.iloc[i].title
    dist, ind = tree.query(encoded_vects[i:i+1], k=3)  # Find nearest neighbors

    # Access nearest neighbors and calculate cosine similarity
    nearest_titles = [df_titles.iloc[j].title for j in ind[0]]
    similar_titles = []
    for neighbor_title in nearest_titles:
      if neighbor_title != current_title:  # Avoid self-comparison
        similarity_score = cosine_similarity(encoded_vects[i], encoded_vects[ind[0][nearest_titles.index(neighbor_title)]])
        if similarity_score > threshold:
          similar_titles.append(neighbor_title)

    similar_titles_dict[current_title] = similar_titles
  return similar_titles_dict

# Find similar titles
similar_titles_dict = find_similar_titles(df_english_titles)

# Print results
for title, similar_titles in similar_titles_dict.items():
  if similar_titles:
    print(f"Similar titles for '{title}':")
    all_similar_titles.append({"title": title, "similar_titles": similar_titles})
    for similar_title in similar_titles:
      print(f"- {similar_title}")

all_similar_titles=pd.DataFrame(all_similar_titles)
all_similar_titles.to_csv(os.path.join(directory, '50k_sample_similar_titles_byBERT.csv'), index=False)

In [ ]:
#all_similar_titles = []

## Find similar titles
#similar_titles_dict = find_similar_titles(df_english_titles)

## Extract similar titles and append to the list
#for title, similar_titles in similar_titles_dict.items():
#  if similar_titles:
#    all_similar_titles.append({"title": title, "similar_titles": similar_titles})

#all_similar_titles=pd.DataFrame(all_similar_titles)
#all_similar_titles.to_csv(os.path.join(directory, '50k_sample_similar_titles_byBERT.csv'), index=False)

# from built-in function

In [5]:
%%time
duplicate_rows = df_english_titles[df_english_titles.duplicated(subset='title')]
print(duplicate_rows)

print("===========================")
len(duplicate_rows)/len(df_english_titles)

                                                                                                                                               title
3631   Book Reviews                                                                                                                                 
3991   Calcutta Diary                                                                                                                               
6105   England and Wales                                                                                                                            
7426   Book Reviews                                                                                                                                 
7607   Defense Briefs                                                                                                                               
7968   Book Reviews                                                                                       

0.0026722215208517443